In [1]:
#From SAV to LOVE

# HW№7

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

## загрузка даты

In [3]:
# MNIST 
batch_size = 64
(x_train, _), (x_test, _) = keras.datasets.fashion_mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_digits = all_digits.astype("float32") / 255
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
dataset = tf.data.Dataset.from_tensor_slices(all_digits)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(32)

## Строим discriminator (расширим кол-во слоев, чтобы поднять качество discriminator)

размер карты 28x28 и бинарная классификация (настоящее изображение или генерировано).

In [4]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),         
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

discriminator.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 128)         147584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 4, 4, 128)         0         
_________________________________________________________________
global_max_pooling2d (Global (None, 128)             

## Строим generator (расширим кол-во слоев)

обратное по отношению к дискриминатору преобразование, меняем `Conv2D` на `Conv2DTranspose` .

In [5]:
latent_dim = 128

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        # строим размер входного вектора 7x7x128 map
        layers.Dense(7 * 7 * 128),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, 128)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              809088    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 6272)              0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 256)       524544    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 28, 28, 256)       0 

## Класс со своим этапом обучения `train_step`

In [6]:

class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        if isinstance(real_images, tuple):
            real_images = real_images[0]
        # берем случайный пример из скрытого пространства
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Строим по нему фейковое изображение
        generated_images = self.generator(random_latent_vectors)

        # собрали с реальным в текзор
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # задаем метки 1 и 0 соответственно
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Добавляем шум !!!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # учим discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        #Выбрали случайный пример в скрытом пространстве
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # собрали метки реальных изображений
        misleading_labels = tf.zeros((batch_size, 1))

        # Учим generator !
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))
        return {"d_loss": d_loss, "g_loss": g_loss}


## Callback для сохранения изображений по ходу обучения

In [7]:

class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=12, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.preprocessing.image.array_to_img(generated_images[i])
            img.save("generated_img_{i}_{epoch}.png".format(i=i, epoch=epoch))


## Учим end-to-end модель, увеличим кол-во эпох, чтобы повысить ее качество

In [8]:
epochs = 12

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

gan.fit(
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=12, latent_dim=latent_dim)]
)

Epoch 1/12
1094/1094 [==============================] - 63s 55ms/step - d_loss: 0.3094 - g_loss: 8.4329
Epoch 2/12
1094/1094 [==============================] - 59s 54ms/step - d_loss: 0.2801 - g_loss: 3.6199
Epoch 3/12
1094/1094 [==============================] - 58s 53ms/step - d_loss: 0.6314 - g_loss: 1.1984
Epoch 4/12
1094/1094 [==============================] - 59s 54ms/step - d_loss: 0.5736 - g_loss: 1.2453
Epoch 5/12
1094/1094 [==============================] - 58s 53ms/step - d_loss: 0.5755 - g_loss: 1.2174
Epoch 6/12
1094/1094 [==============================] - 58s 53ms/step - d_loss: 0.5934 - g_loss: 1.1337
Epoch 7/12
1094/1094 [==============================] - 60s 55ms/step - d_loss: 0.6126 - g_loss: 1.1217
Epoch 8/12
1094/1094 [==============================] - 60s 55ms/step - d_loss: 0.6161 - g_loss: 1.0600
Epoch 9/12
1094/1094 [==============================] - 59s 54ms/step - d_loss: 0.6262 - g_loss: 1.0297
Epoch 10/12
1094/1094 [==============================] - 59s 54m

In [9]:
gan.fit(
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=12, latent_dim=latent_dim)]
)

Epoch 1/12
1094/1094 [==============================] - 59s 54ms/step - d_loss: 0.5837 - g_loss: 1.1130
Epoch 2/12
1094/1094 [==============================] - 58s 53ms/step - d_loss: 0.5668 - g_loss: 1.1508
Epoch 3/12
1094/1094 [==============================] - 58s 53ms/step - d_loss: 0.5542 - g_loss: 1.2175
Epoch 4/12
1094/1094 [==============================] - 58s 53ms/step - d_loss: 0.5472 - g_loss: 1.1789
Epoch 5/12
1094/1094 [==============================] - 59s 54ms/step - d_loss: 0.5422 - g_loss: 1.2658
Epoch 6/12
1094/1094 [==============================] - 60s 55ms/step - d_loss: 0.5371 - g_loss: 1.2102
Epoch 7/12
1094/1094 [==============================] - 59s 54ms/step - d_loss: 0.5312 - g_loss: 1.2403
Epoch 8/12
1094/1094 [==============================] - 59s 54ms/step - d_loss: 0.5249 - g_loss: 1.3316
Epoch 9/12
1094/1094 [==============================] - 59s 54ms/step - d_loss: 0.5232 - g_loss: 1.3044
Epoch 10/12
1094/1094 [==============================] - 60s 55m

**Посмотрим детальный переход**

Display the last generated images:

In [10]:
from IPython.display import Image, display

display(Image("generated_img_0_2.png"))
display(Image("generated_img_1_2.png"))
display(Image("generated_img_2_2.png"))
display(Image("generated_img_3_2.png"))
display(Image("generated_img_4_2.png"))
display(Image("generated_img_5_2.png"))
display(Image("generated_img_6_2.png"))
display(Image("generated_img_7_2.png"))
display(Image("generated_img_8_2.png"))
display(Image("generated_img_9_2.png"))
display(Image("generated_img_10_2.png"))
display(Image("generated_img_11_2.png"))

In [11]:
display(Image("generated_img_2_0.png"))
display(Image("generated_img_2_1.png"))
display(Image("generated_img_2_2.png"))
display(Image("generated_img_2_3.png"))
display(Image("generated_img_2_4.png"))
display(Image("generated_img_2_5.png"))
display(Image("generated_img_2_6.png"))
display(Image("generated_img_2_7.png"))
display(Image("generated_img_2_8.png"))
display(Image("generated_img_2_9.png"))
display(Image("generated_img_2_10.png"))
display(Image("generated_img_2_11.png"))

Наглядно видно как перетекают изображения и увеличивается качество.